In [1]:
import os
from sklearn.model_selection import train_test_split
datahigh = os.listdir("simple_images/high_traffic")
trainhigh, validhigh = train_test_split(datahigh, test_size=0.2, random_state=1)

In [2]:
datalow = os.listdir("simple_images/low_traffic")
trainlow, validlow = train_test_split(datalow, test_size=0.2, random_state=1)

In [3]:
print("Length of High Traffic training dataset  is ", len(trainhigh))
print("Length of High Traffic validation dataset  is ", len(validhigh))
print()
print("Some random images names are")
print(trainhigh[10:20])
print(validhigh[15:25])

Length of High Traffic training dataset  is  160
Length of High Traffic validation dataset  is  40

Some random images names are
['high traffic_98.jpeg', 'huge traffic_3.jpeg', 'huge traffic_26.jpeg', 'high traffic_57.jpeg', 'high traffic_39.jpeg', 'huge traffic_80.jpeg', 'huge traffic_82.jpeg', 'huge traffic_14.jpeg', 'huge traffic_99.jpeg', 'huge traffic_43.jpeg']
['huge traffic_25.jpeg', 'huge traffic_62.jpeg', 'high traffic_40.jpeg', 'huge traffic_41.jpeg', 'high traffic_62.jpeg', 'high traffic_55.jpeg', 'high traffic_23.jpeg', 'high traffic_49.jpeg', 'high traffic_94.jpeg', 'high traffic_37.jpeg']


In [4]:
print("Length of Low Traffic training dataset  is ", len(trainlow))
print("Length of Low Traffic validation dataset  is ", len(validlow))
print()
print("Some random images names are")
print(trainlow[10:20])
print(validlow[15:25])

Length of Low Traffic training dataset  is  172
Length of Low Traffic validation dataset  is  44

Some random images names are
['less traffic_9.jpeg', 'low traffic_51.jpeg', 'low traffic_28.jpeg', 'low traffic_59.png', 'low traffic_16.jpeg', 'low traffic_84.png', 'low traffic_91.jpeg', 'low traffic_58.jpeg', 'low traffic_33.jpeg', 'low traffic_22.jpeg']
['less traffic_37.jpeg', 'low traffic_30.png', 'low traffic_15.jpeg', 'less traffic_49.jpeg', 'low traffic_53.png', 'low traffic_54.jpeg', 'less traffic_44.jpeg', 'low traffic_59.jpeg', 'low traffic_73.jpeg', 'low traffic_92.jpeg']


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 120 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        'split_images/train/',  # This is the source directory for training images
        classes = ['high_traffic', 'low_traffic'],
        target_size=(200, 200),  # All images will be resized to 200x200
        batch_size=120,
        # Use binary labels
        class_mode='binary')

# Flow validation images in batches of 19 using valid_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        'split_images/valid/',  # This is the source directory for training images
        classes = ['high_traffic', 'low_traffic'],
        target_size=(200, 200),  # All images will be resized to 200x200
        batch_size=19,
        # Use binary labels
        class_mode='binary',
        shuffle=False)

Found 332 images belonging to 2 classes.
Found 84 images belonging to 2 classes.


In [7]:
import tensorflow as tf
import numpy as np
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

In [8]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (200,200,3)), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 120000)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               15360128  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 15,360,257
Trainable params: 15,360,257
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [11]:
history = model.fit(train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 16s 2s/step - loss: 29.2949 - accuracy: 0.5296 - val_loss: 17.8310 - val_accuracy: 0.5238


In [12]:
model.evaluate(validation_generator)

5/5 [==============================] - 3s 668ms/step - loss: 17.8310 - accuracy: 0.5238


[17.83102035522461, 0.523809552192688]

In [13]:
#just an effort to improve accuracy

STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size
validation_generator.reset()
preds = model.predict(validation_generator,
                      verbose=1)

5/5 [==============================] - 3s 672ms/step


In [14]:
fpr, tpr, _ = roc_curve(validation_generator.classes, preds)

In [15]:
roc_auc = auc(fpr, tpr)

https://colab.research.google.com/drive/1nseete5huZlWM7Ak0qL-T75Dbk0mdr-Z?usp=sharing#scrollTo=DoWp43WxJDNT